# Project 3


# Conversations Toxicity Detection

## Preproccesing and CuDNNLSTM Model

Jigsaw Unintended Bias in Toxicity Classification 

Detect toxicity across a diverse range of conversations


https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data#


# Import Libraries

In [34]:
import gc
import re
import operator 

import numpy as np
import pandas as pd

from gensim.models import KeyedVectors

from sklearn import model_selection

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import LeakyReLU, CuDNNLSTM
from keras.optimizers import RMSprop, Adam
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping

import seaborn as sns

In [17]:
import os
print(os.listdir("../input"))

['outprocess3', 'jigsaw-unintended-bias-in-toxicity-classification', 'fasttext-crawl-300d-2m']


# Import Data


In [18]:
train = pd.read_csv("../input/outprocess3/train_preprocess2.csv")
test = pd.read_csv("../input/outprocess3/test_preprocess2.csv")
print("Train shape : ",train.shape)
test.head()


Train shape :  (1804874, 2)


,id,comment_text
0,7000000,Jeff Sessions is another one of trump is Orwel...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it will not . That ' s just wishful thinkin...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


# Further Preparation

In [19]:
train_orig = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")
train_orig.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [20]:
train = pd.concat([train,train_orig[['target']]],axis=1)
train.head()

,id,comment_text,target
0,59848,"This is so cool . it is like , ' would you wan...",0.000000
1,59849,Thank you ! ! This would make my life a lot le...,0.000000
2,59852,This is such an urgent design problem ; kudos ...,0.000000
3,59855,Is this something I ' ll be able to install on...,0.000000
4,59856,haha you guys are a bunch of losers .,0.893617


In [21]:
del(train_orig)
gc.collect()

490

Convert target to binary flag

In [22]:
train['target'] = np.where(train['target'] >= 0.5, True, False)

Split into train/validation sets

In [23]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.1)
print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))

1624386 train comments, 180488 validate comments


In [24]:
train_df.head()

,id,comment_text,target
1525131,5987739,"So , the Globe journalist used Gerrymandering ...",False
104146,369546,You want to tell that to every college student...,False
1757018,6275451,There was no informed consent given to search ...,False
580114,952106,Would these protesters feel the same if they o...,False
490475,845174,It is also a state mandate in a very repressiv...,False


Tokenize the text

In [31]:
train_df['comment_text'].describe()

count         1624368
unique        1600928
top       Well said .
freq              176
Name: comment_text, dtype: object

In [36]:
MAX_NUM_WORDS = 100000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

# Create a text tokenizer.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[['comment_text']])

# All comments must be truncated or padded to be the same length.
MAX_SEQUENCE_LENGTH = 256
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

Create our embedding matrix

In [37]:
gc.collect()

577

In [39]:
EMBEDDINGS_DIMENSION = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,EMBEDDINGS_DIMENSION))

In [38]:
ft_common_crawl = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'
embeddings_index = KeyedVectors.load_word2vec_format(ft_common_crawl)

In [40]:
num_words_in_embedding = 0

for word, i in tokenizer.word_index.items():
    if word in embeddings_index.vocab:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector        
        num_words_in_embedding += 1

In [42]:
train_text = pad_text(train_df[[TEXT_COLUMN]], tokenizer)
train_labels = train_df[[TOXICITY_COLUMN]]
validate_text = pad_text(validate_df[[TEXT_COLUMN]], tokenizer)
validate_labels = validate_df[[TOXICITY_COLUMN]]

In [43]:
gc.collect()

162

# Model Architecture

Adding dropout / 1d conv / concatenated poolings based on the architecture presented @ https://www.kaggle.com/tunguz/bi-gru-cnn-poolings-gpu-kernel-version

Now based on: https://www.kaggle.com/samarthsarin/toxication-with-embeddings-and-keras-lstm

In [ ]:
NODES = 64
vocab_size = len(tokenizer.word_index) + 1


model = Sequential()

model.add(Embedding(vocab_size,EMBEDDINGS_DIMENSION,input_length = MAX_SEQUENCE_LENGTH,weights = [embedding_matrix],trainable = False))

model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Conv1D(64,7,padding='same'))
model.add(GlobalAveragePooling1D())

model.add(Dense(128))
model.add(LeakyReLU())

model.add(Dense(NODES,activation = 'relu'))

model.add(Dense(1,activation = 'sigmoid'))

model.summary()


In [ ]:
model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics = ['accuracy'])

# Model Training

In [ ]:
BATCH_SIZE = 1024
NUM_EPOCHS = 100

In [ ]:
model.fit(
    train_text,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=(validate_text, validate_labels),
    callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)])


# Predict & Submit

Let's submit this as our first submission. Once we have a reasonable pipeline setup, we can move on to looking at the competition metric in more detail.

In [ ]:
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv', index_col='id')
submission['prediction'] = model.predict(pad_text(test[TEXT_COLUMN], tokenizer))
submission.reset_index(drop=False, inplace=True)
submission.head()

In [ ]:

submission.to_csv('submission.csv', index=False)